In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

In [ ]:
train_data = pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv')

In [ ]:
print(f"Train_data len : {train_data.shape[0]}")

In [ ]:
test_data = pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Test.csv')

In [ ]:
print(f"Test_data len : {test_data.shape[0]}")

In [ ]:
combined_data = train_data.append([test_data], ignore_index=True)
combined_data

In [ ]:
# removing punctuation marks
combined_data['cleaned_data'] = combined_data['text'].str.replace("[^a-zA-Z#]", " ")
print(combined_data)

In [ ]:
# Removing non english words less than 3
combined_data['cleaned_data'] = combined_data['cleaned_data'].apply(lambda x : ' '.join(w for w in x.split() if len(w)>3))
combined_data

In [ ]:
# converting upper case to lower case characters.
combined_data['cleaned_data'] = combined_data['cleaned_data'].str.lower()
combined_data

In [ ]:
# removing Stop words
english_stopwords = stopwords.words("english")
combined_data['cleaned_data'] = combined_data['cleaned_data'].apply(lambda x : " ".join(w for w in x.split() if w not in english_stopwords))
combined_data

In [ ]:
# Tokenizing the cleaned data.
tokenised_data = combined_data['cleaned_data'].apply(lambda x : x.split())
tokenised_data

In [ ]:
# stemming 
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenised_data = tokenised_data.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenised_data.head()

for i in range(len(tokenised_data)):
    tokenised_data[i] = ' '.join(tokenised_data[i])

combined_data['cleaned_data'] = tokenised_data

In [ ]:
# word count
combined_data['word_count']=combined_data['cleaned_data'].str.lower().str.len()
combined_data

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(combined_data['word_count'],shade=True,color='r').set_title('Kernel Distribution of Number of words')

In [ ]:
normal_words = ' '.join([text for text in combined_data['cleaned_data'][combined_data['label']== 0]])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(normal_words)

plt.figure(figsize=(40,30))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
nagative_words = ' '.join([text for text in combined_data['cleaned_data'][combined_data['label']== 1]])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(nagative_words)

plt.figure(figsize=(40,30))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
tfidf_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
tf_idf= tfidf_vectorizer.fit_transform(combined_data['cleaned_data'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_tfidf = tf_idf[:40000,:]
test_tfidf = tf_idf[40000:,:]

xtrain_tf_idf, xvalid_tf_idf, ytrain, yvalid = train_test_split(train_tfidf ,train_data['label'],random_state=42, test_size=0.3)

lreg = LogisticRegression()
lreg.fit(xtrain_tf_idf, ytrain)

prediction = lreg.predict_proba(xvalid_tf_idf)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)
f1_score(yvalid, prediction_int)